In [ ]:
import pandas as pd
import numpy as np
import pickle
import operator
import networkx as nx
import os
import csv
import yaml
from st_network2 import Many2Many
from trip_prediction import trip_prediction

from graph_coarsening import graph_coarsening
from utils import *

In [3]:
with open("config_m2m_gc.yaml", "r+") as fopen:
    config = yaml.load(fopen, Loader=yaml.FullLoader)
config["FIXED_ROUTE"] = True

In [4]:
Route_fr_disagg = load_FR_disagg(config)
Route_fr_disagg = None
# Load mode choice input data
(
    per_time,
    per_dist,
    per_emp,
    mdot_dat,
    dests_geo,
    D,
) = load_mc_input(config)

# load ID converter between GEOID and zone_id
id_converter = pickle.load(open(config["id_converter"], "rb"))

In [5]:
(
    trips_dict_pk,
    trips_dict_op,
    transit_trips_dict_pk,
    transit_trips_dict_op,
) = trip_prediction(
    id_converter, per_dist, per_emp, mdot_dat, dests_geo, D, per_time=per_time
)

In [6]:
(
    N_n,
    L_l,
    C_l,
    K_k,
    w_out,
    w_in,
    w_f,
) = get_link_cost(trips_dict_pk, Route_fr_disagg, config)
(_, _, P_N, P_L, _) = graph_coarsening(N_n, L_l, C_l, K_k, config)

Academic license - for non-commercial use only - expires 2021-06-19
Using license file C:\Users\SQwan\gurobi.lic
Parameter TimeLimit unchanged
   Value: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter MIPGap to 0.2
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8784 rows, 5240 columns and 26160 nonzeros
Model fingerprint: 0xe9f9c078
Variable types: 0 continuous, 5240 integer (5240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 88.0000000
Presolve removed 1760 rows and 880 columns
Presolve time: 0.06s
Presolved: 7024 rows, 4360 columns, 22640 nonzeros
Variable types: 0 continuous, 4360 integer (4360 binary)

Root relaxation: objective 2.774014e+02, 5423 iterat

In [7]:
# Record current agg2disagg of those bus-visited zones
bus_zone = set(stop[2] for _, route in Route_fr_disagg.items() for stop in route)
# Update agg_2_disagg_id and disagg_2_agg_id
agg_2_disagg_id = dict()

for idx, c in enumerate(P_N.keys()):
    agg_2_disagg_id[idx] = P_N[c]
# P_N excludes the zones with bus service, need to add those excluded zones
# into agg_2_disagg_id
for idx, node in enumerate(bus_zone):
    new_part_id = len(P_N) + idx
    agg_2_disagg_id[new_part_id] = [node]


disagg_2_agg_id = {
    n: partition for partition, nodes in agg_2_disagg_id.items() for n in nodes
}

# Update config dictionary: number of partitions of aggregated network
config["S"] = len(agg_2_disagg_id)
print(config["S"])

38


In [8]:
# Load neighbor nodes information of disaggregated zones
ctr_disagg = load_neighbor_disagg(config)
# Update ctr, i.e. station neighbor info of aggregated network
ctr = update_ctr_agg(ctr_disagg, disagg_2_agg_id)

In [16]:
file_dir = config["m2m_output_loc"] + "init_graph\\"
if not os.path.exists(file_dir):
    os.makedirs(file_dir)
pickle.dump(ctr, open(file_dir + "ctr_agg.p", "wb"))
pickle.dump(ctr_disagg, open(file_dir + "ctr_disagg.p", "wb"))
pickle.dump(agg_2_disagg_id, open(file_dir + "agg_2_disagg_id.p", "wb"))
pickle.dump(disagg_2_agg_id, open(file_dir + "disagg_2_agg_id.p", "wb"))